## Avec Pytorch

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle 
from PIL import Image 
import numpy as np
import torch
from torchvision import models, utils
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [4]:
# Charger l'image à l'aide de PIL
img = Image.open("cat.png")

# Supprimer le canal d'opacité (alpha) si présent
if img.mode == "RGBA":
    img = img.convert("RGB")

# Redimensionner l'image en utilisant la méthode bilinéaire
img = img.resize((224, 224), Image.BILINEAR)

# Convertir l'image en tableau numpy et normaliser les valeurs
img = transforms.ToTensor()(img)
img = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)(img)

# Ajouter une dimension au tenseur d'image
x = img.unsqueeze(0)

# Charger le modèle pré-entraîné VGG16_bn
model = models.vgg16_bn(pretrained=True)

# Mettre le modèle en mode d'évaluation
model.eval()

# Utiliser un GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
x = x.to(device)

# Passer le tenseur d'image x à travers le modèle pour effectuer un calcul forward
with torch.no_grad():
    y = model(x)

# Charger les étiquettes des classes ImageNet
with open("imagenet_classes.txt") as f:
    labels = [line.strip() for line in f.readlines()]

# Obtenir les indices des classes prédites et les probabilités correspondantes
probs, indices = torch.topk(torch.softmax(y, dim=1), k=5)

# Afficher les prédictions de classe avec les probabilités correspondantes
for i in range(probs.size(1)):
    class_idx = indices[0, i].item()
    class_prob = probs[0, i].item()
    class_label = labels[class_idx]
    print(f"Classe : {class_label}, Probabilité : {class_prob:.4f}")


Classe : 277, red_fox, Probabilité : 0.5017
Classe : 278, kit_fox, Probabilité : 0.4222
Classe : 281, tabby, Probabilité : 0.0726
Classe : 283, Persian_cat, Probabilité : 0.0015
Classe : 900, water_tower, Probabilité : 0.0003


## Avec Tensorflow

In [5]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

In [6]:
# Load the pre-trained VGG16 model
model = VGG16(weights='imagenet')

# Load and preprocess your image
img = image.load_img("cat.png", target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Make predictions on the image
preds = model.predict(x)
decoded_preds = decode_predictions(preds, top=5)[0]

# Print the top 5 predictions print("Predictions:")
for _, label, probability in decoded_preds:
  print(f"{label}: {probability:.4f}")

35363/35363 [==============================] - 0s 0us/step
tiger_cat: 0.5739
tabby: 0.1939
Egyptian_cat: 0.1769
lynx: 0.0481
feather_boa: 0.0012
